In [1]:
from utils.utils import *

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


## Epidemics

### GKAN

In [7]:
model_path = './saved_models_optuna/model-epidemics-gkan/epidemics-new-data/2/gkan'

In [8]:
pysr_model = lambda : get_pysr_model(
    model_selection="score",
    random_state = 0,
    deterministic=True,
    parallelism='serial',
    n_iterations=150
)

symb_model_black_box = fit_black_box_from_kan(
    model_path=model_path,
    n_g_hidden_layers=2,
    n_h_hidden_layers=2,
    theta=-np.inf,
    pysr_model=pysr_model,
    sample_size=9000,
    message_passing=False
)

/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/pysr/sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/pysr/sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


In [9]:
symb_model_black_box

\sum_{j}( 0.500805746365327*x_j*(0.9996049 - x_i)) - 0.49989313*x_i

### MLP-based baseline

In [11]:
model_path_mpnn = './saved_models_optuna/model-epidemics-mpnn/epidemics-new-data/1'

In [12]:
pysr_model = lambda : get_pysr_model(
    model_selection="score",
    random_state = 0,
    deterministic=True,
    parallelism='serial',
    n_iterations=150
)

mpnn_symb = fit_mpnn(
    model_path=model_path_mpnn,
    pysr_model=pysr_model,
    sample_size=9000,
    message_passing=False
)


/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/pysr/sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/pysr/sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


In [13]:
mpnn_symb

\sum_{j}( x_i*(0.49942818 - 0.49865368*x_j)) - 0.5019439*x_i

## Kuramoto

### GKAN

In [8]:
model_path = 'saved_models_optuna/model-kuramoto-gkan/kuramoto-new-data/0/gkan'

In [9]:
pysr_model = lambda : get_pysr_model(
    model_selection="score",
    n_iterations=200
)

symb_model_black_box = fit_black_box_from_kan(
    model_path=model_path,
    n_g_hidden_layers=2,
    n_h_hidden_layers=2,
    theta=0.1,
    pysr_model=pysr_model,
    sample_size=8000,
    message_passing=False
)

Pruning node (0,0)


/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/pysr/sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Pruning node (0,0)


/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/pysr/sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


In [10]:
symb_model_black_box

\sum_{j}( -0.49825495*sin(x_i - x_j)) + 0.040180836*x_i + 1.2174591

## Test Experiments

In [2]:
import optuna
from optuna.storages import JournalStorage
from optuna.storages.journal import JournalFileBackend
from experiments.experiments_gkan import ExperimentsGKAN

storage = JournalStorage(JournalFileBackend("optuna_journal_storage.log"))

/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Kuramoto

In [2]:
optuna.delete_study(study_name="model-kuramoto-gkan-test-new-dataset", storage=storage)

In [3]:
config_path = './configs/config_kuramoto.yml'
config = load_config(config_path)


exp = ExperimentsGKAN(
    config=config,
    n_trials=2,
    study_name='test-new-dataset',
    process_id=0,
    store_to_sqlite = False
)

exp.run()


CPU times: user 1e+03 ns, sys: 1 μs, total: 2 μs
Wall time: 5.72 μs
Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


Processing...


Builing the dataset...


Done!
[I 2025-04-07 15:35:26,586] A new study created in Journal with name: model-kuramoto-gkan-test-new-dataset
[I 2025-04-07 15:36:16,853] Trial 0 finished with value: 0.004770759493112564 and parameters: {'lr': 0.004328450221293881, 'lamb': 1.0, 'batch_size': 32, 'use_orig_reg': True, 'lamb_g_net': 0.0015751320499779737, 'lamb_h_net': 0.0002051338263087451, 'grid_size_g_net': 5, 'spline_order_g_net': 1, 'range_limit_g_net': 10, 'mu_1_g_net': 0.8, 'mu_2_g_net': 0.9, 'hidden_dim_g_net': 1, 'grid_size_h_net': 10, 'spline_order_h_net': 3, 'range_limit_h_net': 6, 'mu_1_h_net': 0.6, 'mu_2_h_net': 0.6, 'hidden_dim_h_net': 1}. Best is trial 0 with value: 0.004770759493112564.
[I 2025-04-07 15:37:02,277] Trial 1 finished with value: 0.002222407143563032 and parameters: {'lr': 0.0077901431262762414, 'lamb': 1.0, 'batch_size': 32, 'use_orig_reg': True, 'lamb_g_net': 0.0016738085788752138, 'lamb_h_net': 0.00019010245319870352, 'grid_size_g_net': 6, 'spline_order_g_net': 2, 'range_limit_g_net': 

### Test traffic data

In [6]:
optuna.delete_study(study_name="model-metrla-gkan-test-new-dataset", storage=storage)

In [3]:
config_path = './configs/config_metrla_gkan.yml'
config = load_config(config_path)

In [4]:
exp = ExperimentsGKAN(
    config=config,
    n_trials=2,
    study_name='test-new-dataset',
    process_id=0,
    store_to_sqlite = False
)

Processing...


Builing the dataset...


download: 13.1MB [00:01, 6.99MB/s]                            
Done!


In [5]:
exp.epochs = 10
exp.run()

[I 2025-04-07 17:06:38,093] A new study created in Journal with name: model-metrla-gkan-test-new-dataset
[I 2025-04-07 17:07:10,833] Trial 0 finished with value: 0.07330862432718277 and parameters: {'lr': 0.0010253509690168502, 'lamb': 0.0, 'batch_size': 32, 'use_orig_reg': False, 'lamb_g_net': 0.006351221010640704, 'lamb_h_net': 0.0008471801418819979, 'grid_size_g_net': 8, 'spline_order_g_net': 1, 'range_limit_g_net': 1, 'mu_1_g_net': 0.5, 'mu_2_g_net': 1.0, 'hidden_dim_g_net': 2, 'grid_size_h_net': 9, 'spline_order_h_net': 1, 'range_limit_h_net': 3, 'mu_1_h_net': 0.9, 'mu_2_h_net': 0.6, 'hidden_dim_h_net': 1}. Best is trial 0 with value: 0.07330862432718277.
[I 2025-04-07 17:07:54,804] Trial 1 finished with value: 0.0793130099773407 and parameters: {'lr': 0.0003126102910311062, 'lamb': 0.0, 'batch_size': 32, 'use_orig_reg': False, 'lamb_g_net': 1.6480446427978994e-05, 'lamb_h_net': 0.0001256104370001356, 'grid_size_g_net': 7, 'spline_order_g_net': 1, 'range_limit_g_net': 2, 'mu_1_g_n